In [1]:
# IMPORTS FOR NGENET AND DEFORMATION PYRAMID
import os
from glob import glob
from pickle import load, dump
import argparse

import numpy as np
import open3d as o3d
import torch
import trimesh as trm
import yaml
import time
from scipy.spatial.distance import cdist
from statistics import mean
from easydict import EasyDict as edict

from deformationpyramid.model.geometry import *
from deformationpyramid.model.loss import compute_truncated_chamfer_distance
from deformationpyramid.model.registration import Registration
from tqdm import tqdm
from ngenet.data import EarDataset, EarDatasetTest, get_dataloader
from ngenet.models import NgeNet, architectures, vote
from ngenet.utils import (decode_config, execute_global_registration, get_blue,
                          get_correspondences, get_yellow, npy2feat, npy2pcd, pcd2npy,
                          setup_seed, to_tensor, vis_plys)
from deformationpyramid.utils.benchmark_utils import setup_seed
from deformationpyramid.utils.tiktok import Timers

In [2]:
def join(loader, node):
    seq = loader.construct_sequence(node)
    return '_'.join([str(i) for i in seq])
yaml.add_constructor('!join', join)

def checkIPython():
    try:
        get_ipython().__class__.__name__
        return True
    except:
        return False

if checkIPython(): # Checks if running in IPython notebook. If running by CLI, argparse is used
    class config:
        pass
    args = config()
    args.data_root = 'mesh_dataset/ear_dataset/'
    args.dataset_split = 'test'
    args.oct_data_root = 'mesh_dataset/oct_outputs/*.npy'
    args.checkpoint = 'trainResults/eardataset_nonrigid_randrot_pretrained_eardrum_large_ds/checkpoints/best_recall.pth'
    #args.checkpoint = 'trainResults/trained_03_13_100k_randomsplit_0/checkpoints/best_recall.pth'
    args.vis = False
    args.no_cuda = False
    args.use_real = True

    args.ngenet_config_path = 'config/eardataset_ngenet.yaml'
    args.ndp_config_path = 'config/NDP.yaml'
else:
    parser = argparse.ArgumentParser()

    parser.add_argument('--data_root', type=str, default='mesh_dataset/ear_dataset/', help='root of synthetic dataset')
    parser.add_argument('--dataset_split', type=str, default='val', help='which of the splits should be used as synthetic dataset')
    parser.add_argument('--oct_data_root', type=str, default='mesh_dataset/oct_outputs/*.npy', help='glob specification of all oct scans to test (convert to .npy first)')
    parser.add_argument('--checkpoint', type=str, default='trainResults/eardataset_nonrigid_randrot_pretrained_eardrum_large_ds/checkpoints/best_loss.pth', help='path to NgeNet checkpoint')
    parser.add_argument('--ngenet_config_path', type=str, default='config/eardataset.yaml', help='which configuration file to use for NgeNet')
    parser.add_argument('--ndp_config_path', type=str, default='config/NDP.yaml', help='which configuration file to use for NDP')
    parser.add_argument('--use_real', action='store_true', default=False, help='decide wheather to run the test on the oct scans')
    parser.add_argument('--vis', action='store_true',  default=False, help='visualize output while running in an extra window')
    parser.add_argument('--no_cuda', action='store_true',  default=False, help='disable cuda (cpu only)')

    args = parser.parse_args()


In [3]:
setup_seed(22)
config = decode_config(args.ngenet_config_path)
config = edict(config)
config.architecture = architectures[config.dataset]
config.num_workers = 0

diome_path = 'mesh_dataset/DIOME_FanShapeCorr/'

test_dataset = EarDataset(
    root=args.data_root,
    noisy_intra=config.noisy_intra,
    split=args.dataset_split,
    aug=False,
    overlap_radius=config.overlap_radius
)

test_dataset_real = EarDatasetTest(
    test_path=diome_path,
    root=args.data_root,
    noisy_intra=config.noisy_intra,
    split='test',
    aug=False,
    overlap_radius=config.overlap_radius
)

metadata = test_dataset.metadata

test_dataloader, neighborhood_limits = get_dataloader(
    config=config,
    dataset=test_dataset_real if args.use_real else test_dataset,
    batch_size=config.batch_size,
    num_workers=config.num_workers,
    shuffle=False,
    neighborhood_limits=None
)

with open(args.ndp_config_path,'r') as f:
    p_config = yaml.load(f, Loader=yaml.Loader)

p_config = edict(p_config)

In [4]:
with open('mesh_dataset/landmarks/landmarks.pkl', 'rb') as f:
    landmarks = load(f)

def registration_cd(points_pred, points_tgt, corrs, pred_T=np.eye(4)):
    '''

    :param points_pred: (n, 3)
    :param points_tgt: (m, 3)
    :return: float
    '''
    if len(corrs) != 0:
        R, t = pred_T[:3, :3], pred_T[:3, 3]
        points_pred = points_pred @ R.T + t
        points_pred = torch.tensor(points_pred[corrs]).T.unsqueeze(0).float()
        points_tgt = torch.tensor(points_tgt).T.unsqueeze(0).float()
        cd = compute_truncated_chamfer_distance(points_pred, points_tgt, trunc=1e+9)
        cd = cd.item()
    else:
        cd = 1e+4
    return cd

def mean_displacement_error(dis_pred, dis_gt):
    return np.linalg.norm(dis_pred-dis_gt, axis=1).mean()

def denorm(arr, metadata):
    return arr * metadata['std'] + metadata['mean']

def landmark_loss(pred, intra):
    assert len(pred) == len(intra), 'len(pred) != len(intra)'
    l = []
    single_loss = {}
    segments = list(intra.keys())
    for seg in range(len(pred)):
        mat = cdist(pred[seg], intra[segments[seg]]).min(0)
        
        l.append(mat.mean())
        single_loss[segments[seg]] = mat.mean()
    return sum(l)/len(l), single_loss

In [5]:
model_rigid = NgeNet(config)
use_cuda = not args.no_cuda
if use_cuda:
    model = model_rigid.cuda()
    if args.checkpoint != None:
        model_rigid.load_state_dict(torch.load(args.checkpoint))
    p_config.device = torch.cuda.current_device()
else:
    model_rigid.load_state_dict(
        torch.load(args.checkpoint, map_location=torch.device('cpu')))
    config.device = torch.device('cpu')
model_rigid.eval()

fmr_threshold = 0.05
rmse_threshold = 0.2
inlier_ratios, mutual_inlier_ratios = [], []
mutual_feature_match_recalls, feature_match_recalls = [], []
transformations = []
nonregistered_cd_l, registered_cd_l, mean_displacement_error_l, landmark_loss_l, ngenet_cd_l = [], [], [], [], []
overlap_scores, wall_time_models = [], []
displ_ngenet, displ = [], []
mdes, visible_point_ratio = [], []
uncertainty_scores = []
side, status = [], []
len_coors_l = []

dist_thresh_maps = {
    '10000': config.first_subsampling_dl,
    '5000': config.first_subsampling_dl,
    '2500': config.first_subsampling_dl * 1.5,
    '1000': config.first_subsampling_dl * 1.5,
    '500': config.first_subsampling_dl * 1.5,
    '250': config.first_subsampling_dl * 2,
}

model_nonrigid = Registration(p_config)
timer = Timers()

In [6]:
anulus, umbo, malleus_handle, long_process_of_incus, stape = [], [], [], [], []

for pair_ind, inputs in enumerate(tqdm(test_dataloader)):
    t1 = time.time()
    if use_cuda:
        for k, v in inputs.items():
            if isinstance(v, list):
                for i in range(len(v)):
                    inputs[k][i] = inputs[k][i].cuda()
            else:
                inputs[k] = inputs[k].cuda()
    with torch.no_grad():
        batched_feats_h, batched_feats_m, batched_feats_l = model_rigid(inputs)
        stack_points = inputs['points']
        stack_points_raw = inputs['batched_points_raw']
        stack_lengths = inputs['stacked_lengths']
        coords_tgt_full = inputs['points_tgt_full']
        coords_src = stack_points[0][:stack_lengths[0][0]]
        coords_tgt = stack_points[0][stack_lengths[0][0]:]
        coords_src_raw = stack_points_raw[:stack_lengths[0][0]]
        coords_tgt_raw = stack_points_raw[stack_lengths[0][0]:]
        feats_src_h = batched_feats_h[:stack_lengths[0][0]]
        feats_tgt_h = batched_feats_h[stack_lengths[0][0]:]
        feats_src_m = batched_feats_m[:stack_lengths[0][0]]
        feats_tgt_m = batched_feats_m[stack_lengths[0][0]:]
        feats_src_l = batched_feats_l[:stack_lengths[0][0]]
        feats_tgt_l = batched_feats_l[stack_lengths[0][0]:]

        displacement_gt = inputs['transf'][0].detach().cpu().numpy() * metadata['std']

        source_npy = coords_src.detach().cpu().numpy()
        target_npy = coords_tgt.detach().cpu().numpy()

        source_feats_h = feats_src_h[:, :-2].detach().cpu().numpy()
        target_feats_h = feats_tgt_h[:, :-2].detach().cpu().numpy()
        source_feats_m = feats_src_m.detach().cpu().numpy()
        target_feats_m = feats_tgt_m.detach().cpu().numpy()
        source_feats_l = feats_src_l.detach().cpu().numpy()
        target_feats_l = feats_tgt_l.detach().cpu().numpy() 
        
        after_vote = vote(
            source_npy=source_npy, 
            target_npy=target_npy, 
            source_feats=[source_feats_h, source_feats_m, source_feats_l], 
            target_feats=[target_feats_h, target_feats_m, target_feats_l], 
            voxel_size=config.first_subsampling_dl,
            use_cuda=use_cuda)
        source_npy, target_npy, source_feats_npy, target_feats_npy = after_vote

        source, target = npy2pcd(source_npy), npy2pcd(target_npy)
        
        source_feats, target_feats = npy2feat(source_feats_h), npy2feat(target_feats_h)
        pred_T, estimate, result = execute_global_registration(
            source=source,
            target=target,
            source_feats=source_feats,
            target_feats=target_feats,
            voxel_size=dist_thresh_maps['10000']
        )
        
        transformations.append(pred_T)
    
    corrs_pred = np.unique(np.asarray(result.correspondence_set).T[0])
    len_coors_l.append(len(corrs_pred))
    raw_coords_src = torch.tensor(denorm(pcd2npy(estimate), metadata)).to(coords_src.device)
    raw_coords_tgt = torch.tensor(denorm(pcd2npy(target), metadata)).to(coords_tgt.device)

    coords_src_raw = coords_src_raw.cpu().detach().numpy()
    coords_tgt_raw = coords_tgt_raw.cpu().detach().numpy()
    coords_tgt_full = coords_tgt_full.cpu().detach().numpy()
    coords_transformed_src = raw_coords_src.cpu().detach().numpy()

    inds = inputs['inds'][0].cpu().detach().numpy()
    faces = inputs['faces'][0].cpu().detach().numpy()
    
    model_nonrigid.load_pcds(raw_coords_src.float(), raw_coords_tgt.float(), inds=corrs_pred, search_radius=0.0375)
    warped_pcd, hist, iter_cnt, timer = model_nonrigid.register(visualize=args.vis, timer = timer)
    warped_pcd = warped_pcd.cpu().numpy()
    
    
    t2 = time.time()
    wall_time_models.append(t2-t1)
    
    displacement_ngenet = raw_coords_src.cpu().detach().numpy() - coords_src_raw
    displacement_pred = warped_pcd - coords_src_raw
    displ_ngenet.append(displacement_ngenet)
    displ.append(displacement_pred)
    
    if not args.use_real:
        registered_cd = compute_truncated_chamfer_distance(
            torch.tensor(warped_pcd).unsqueeze(0), 
            torch.tensor(coords_tgt_full).unsqueeze(0), 
            trunc=1e+9
        ).item()
        
        nonregistered_cd = compute_truncated_chamfer_distance(
            torch.tensor(coords_src_raw).unsqueeze(0), 
            torch.tensor(coords_tgt_full).unsqueeze(0), 
            trunc=1e+9
        ).item()

        ngenet_cd = compute_truncated_chamfer_distance(
            torch.tensor(coords_transformed_src).unsqueeze(0).float(), 
            torch.tensor(coords_tgt_full).unsqueeze(0), 
            trunc=1e+9
        ).item()

        # MEAN DISPLACEMENT ERROR
        mde = mean_displacement_error(
            displacement_pred, 
            displacement_gt
        )
        mdes.append(mde)

        # UNCERTAINTY PREDICTION
        # based on source target distance
        dist = torch.norm(torch.tensor(warped_pcd).unsqueeze(1) - torch.tensor(coords_tgt_raw).unsqueeze(0), dim=2)
        minimumDistances, _ = dist.min(dim=1)
        minimumDistances = torch.tanh(minimumDistances)
        
        visible_point_ratio.append(coords_tgt.shape[0]/5995)

        l_inds = [v for u, v in landmarks.items()]
        pred_landmarks = [warped_pcd[i] for i in l_inds]
        pre_landmarks = [coords_tgt_full[i] for i in l_inds]
        
        lndmk = landmark_loss(pre_landmarks, pred_landmarks)
        ind = metadata[args.dataset_split][pair_ind].split("/")[1]
        landmark_loss_l.append(lndmk)
        ngenet_cd_l.append(ngenet_cd)
    else:
        
        registered_cd = compute_truncated_chamfer_distance(
            torch.tensor(warped_pcd).unsqueeze(0), 
            torch.tensor(coords_tgt_raw).unsqueeze(0), 
            trunc=1e+9
        ).item()
        
        nonregistered_cd = compute_truncated_chamfer_distance(
            torch.tensor(coords_src_raw).unsqueeze(0), 
            torch.tensor(coords_tgt_raw).unsqueeze(0), 
            trunc=1e+9
        ).item()

        intra_data = test_dataset_real.__getitem__(pair_ind)
        intra_metadata = intra_data['metadata']
        side.append(intra_metadata['patient_info']['side'])
        status.append(intra_metadata['patient_info']['status'])
        
        landmarks_intra = intra_data['landmarks']

        ind = pair_ind
        if landmarks_intra != {}:
            pred_landmarks = [warped_pcd[landmarks[k]] for k, v in landmarks_intra.items()]
            intra_landmarks = {k:v for k, v in landmarks_intra.items()}
            lndmk, single_loss = landmark_loss(pred_landmarks, intra_landmarks)
            landmark_loss_l.append(lndmk)

            segments = list(single_loss.keys())
            if 'anulus' in segments:
                anulus.append(single_loss['anulus'])
            else:
                anulus.append(float('nan'))

            if 'Umbo' in segments:
                umbo.append(single_loss['Umbo'])
            else:
                umbo.append(float('nan'))
            
            if 'malleus handle' in segments:
                malleus_handle.append(single_loss['malleus handle'])
            else:
                malleus_handle.append(float('nan'))

            if 'long process of incus' in segments:
                long_process_of_incus.append(single_loss['long process of incus'])
            else:
                long_process_of_incus.append(float('nan'))

            if 'stape' in segments:
                stape.append(single_loss['stape'])
            else:
                stape.append(float('nan'))
        else:
            anulus.append(float('nan'))
            umbo.append(float('nan'))
            malleus_handle.append(float('nan'))
            long_process_of_incus.append(float('nan'))
            stape.append(float('nan'))
            landmark_loss_l.append(-1)
        mde = -1
    
    overlap = len(target_npy)/len(source_npy)
    overlap_scores.append(overlap)
    registered_cd_l.append(registered_cd)
    nonregistered_cd_l.append(nonregistered_cd)
    mean_displacement_error_l.append(mde)
    
    
    output_mesh = trm.Trimesh(warped_pcd, faces)
    _=output_mesh.export(f'test_output_folder/predictions/prediction_{ind}.stl')
    oct_pcd = o3d.geometry.PointCloud()
    oct_pcd.points = o3d.utility.Vector3dVector(np.array(warped_pcd))
    o3d.io.write_point_cloud(f'test_output_folder1/predictions/prediction_{ind}.ply', oct_pcd)
    oct_pcd = o3d.geometry.PointCloud()
    oct_pcd.points = o3d.utility.Vector3dVector(coords_tgt_raw)
    o3d.io.write_point_cloud(f'test_output_folder1/target shape/target_{ind}.ply', oct_pcd)

    np.save(f'test_output_folder/pred_corrs/pred_corrs_{ind}.npy', np.asarray(result.correspondence_set))
    np.save(f'test_output_folder/ndp_hist/ndp_hist_{ind}.npy', hist)
    np.save(f'test_output_folder/other_scores/overlap_saliency_{ind}.npy', feats_src_h[:, -2].cpu().numpy())

    if pair_ind == -1:
        break

100%|██████████| 43/43 [06:14<00:00,  8.71s/it]


In [8]:
import pandas as pd

results_dict_exvivo = dict(
    nonregistered_cd_l=nonregistered_cd_l,
    registered_cd_l=registered_cd_l,
    mean_displacement_error_l=mean_displacement_error_l,
    overlap_scores=overlap_scores,
    landmark_loss_l=[i if i != -1 else float('nan') for i in landmark_loss_l],
    wall_time_models=wall_time_models,
    ngenet_cd_l=ngenet_cd_l,
    visible_point_ratio=visible_point_ratio,
    displ_ngenet=displ_ngenet,
    displ=displ,
    len_coors_l=len_coors_l,
)


df = pd.DataFrame.from_dict(results_dict_exvivo)
df.to_csv('finalResults/results_exvivo_ngenet_variance.csv')

In [7]:
print('Non-registered cd score:', mean(nonregistered_cd_l))
print('Registered cd score:', mean(registered_cd_l))
print('Overlap of pointclouds:', mean(overlap_scores))
print('Mean displacement error: ', mean(mean_displacement_error_l))
print('Landmark loss:', mean([i for i in landmark_loss_l if i != -1]))
print('Wall time:', mean(wall_time_models), 's')

Non-registered cd score: 7.326125677241835
Registered cd score: 2.2037578405335894
Overlap of pointclouds: 1.0
Mean displacement error:  -1
Landmark loss: 2.0358495961072816
Wall time: 7.6358127871225046 s


In [9]:
colors = np.zeros((5995, 3))
colors[:, 1] = feats_src_h[:, -2].cpu().numpy()

warped_pcds = npy2pcd(raw_coords_src.cpu().numpy())
warped_pcds.colors = o3d.utility.Vector3dVector(colors)
target = npy2pcd(raw_coords_tgt.cpu().numpy())
target.paint_uniform_color([255, 0, 0])
vis_plys([warped_pcds, target], need_color=False)

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


In [8]:
sample_index = 5
print(f'Chamfer distance: {registered_cd_l[sample_index]}')
vis_plys([npy2pcd(warped_pcds[sample_index]), npy2pcd(tgt_raw[sample_index].cpu().numpy())])

Chamfer distance: 1.8108539581298828


NameError: name 'warped_pcds' is not defined

In [ ]:
""" Non-registered cd score: 1.459614505165906
Registered cd score: 0.5679943127659356
Overlap of pointclouds: 0.356882301948132
Mean displacement error:  0.7746529
Landmark loss: 0.4213273387613913
Wall time: 39.942818044655645 s """

In [ ]:
""" DIOME no correction
Non-registered cd score: 4.754870608795521
Registered cd score: 1.6517222634581632
Overlap of pointclouds: 0.5124270225187656
Mean displacement error:  -1
Landmark loss: 2.2849106172687916
Wall time: 6.274375043132088 s 

DIOME with correction
NNon-registered cd score: 4.179325480793798
Registered cd score: 2.3342912876328756
Overlap of pointclouds: 0.3749791492910759
Mean displacement error:  -1
Landmark loss: 1.817811517964032
Wall time: 8.124897069709245 s"""

In [49]:
import pandas as pd

results_dict_invivo = dict(
    nonregistered_cd_l=nonregistered_cd_l,
    registered_cd_l=registered_cd_l,
    overlap_scores=overlap_scores,
    landmark_loss_l=[i if i != -1 else float('nan') for i in landmark_loss_l],
    wall_time_models=wall_time_models,
    displ_ngenet=displ_ngenet,
    displ=displ,
    side=side,
    status=status,
    anulus=anulus,
    umbo=umbo,
    malleus_handle=malleus_handle,
    long_process_of_incus=long_process_of_incus,
    stape=stape
)


df = pd.DataFrame.from_dict(results_dict_invivo)
df.to_csv('finalResults/results_invivo_ngenet2.csv')